In [1]:
import sys
sys.executable = '/home/chao/anaconda3/envs/chemcheck_env/lib/python3.8/site-packages'
python_path = ['/home/chao/cantera/build/python', '/home/chao/RMG-Py', '/home/chao', '/home/chao/anaconda3/envs/chemcheck_env/lib/python38.zip', '/home/chao/anaconda3/envs/chemcheck_env/lib/python3.8', '/home/chao/anaconda3/envs/chemcheck_env/lib/python3.8/lib-dynload', '/home/chao/anaconda3/envs/chemcheck_env/lib/python3.8/site-packages']
sys.path.clear()
for path in python_path:
    sys.path.append(path)

In [2]:
import yaml
import cantera as ct
import numpy as np
from __future__ import division
from __future__ import print_function
import pandas as pd
import time
from textwrap import wrap
import math

import cantera as ct
print('Runnning Cantera version: ' + ct.__version__)

# %matplotlib notebook
import matplotlib.pyplot as plt

Runnning Cantera version: 2.6.0a2


In [3]:
sensitive_rxns = ["H2O2(18) (+M) <=> 2 OH(15) (+M)",
                  "CH4(10) + O2(2) <=> CH3(19) + HO2(17)",
                  "C7H15(686) + H2O2(18) <=> C7H16(1) + HO2(17)",
                  "C2H4(11) + OH(15) <=> C2H3(23) + H2O(8)",
                  "C2H4(11) + HO2(17) <=> C2H5(34) + O2(2)",
                  "C7H15(683) + H2O2(18) <=> C7H16(1) + HO2(17)",
                  "2 HO2(17) <=> H2O2(18) + O2(2)",
                  "HO2(17) + OH(15) <=> H2O(8) + O2(2)",
                  "CH3(19) + HO2(17) <=> CH3O(31) + OH(15)",
                  "CH2O(9) + HO2(17) <=> H2O2(18) + HCO(20)"
                 ]

In [4]:
# gas = ct.Solution('n_heptane/n_heptane_annotated.yaml')
gas = ct.Solution('n_heptane/n_heptane_bm.yaml')

For species HOCHO(59), discontinuity in h/RT detected at Tmid = 1000.0
	Value computed using low-temperature polynomial:  -40.36040333333333
	Value computed using high-temperature polynomial: -38.16784474



In [5]:
sens_indices = []
# for i, rxn in enumerate(gas.reactions()):
#     if rxn.equation in sensitive_rxns:
#         sens_indices.append(i)
#         print(i)
for r in sensitive_rxns:
    duplicate_rxns = []
    for i, rxn in enumerate(gas.reactions()):
        if rxn.equation == r:
            duplicate_rxns.append(i)
    sens_indices.append(duplicate_rxns)
print(sens_indices)

[[21], [80], [871], [243], [249], [917], [19, 20], [18], [96], [63, 64]]


In [6]:
for rxn_idx in sens_indices:
    for i in rxn_idx:
        print(gas.reaction(i).reaction_type)

falloff
elementary
Blowers-Masel
elementary
pressure-dependent-Arrhenius
Blowers-Masel
elementary
elementary
elementary
elementary
elementary
elementary


In [19]:
gas.reaction(249).rates

[(1333.437, Arrhenius(A=2.8e+09, b=-0.447, E=5.29653e+07)),
 (99999.66899999997, Arrhenius(A=340700, b=0.818, E=5.05846e+07)),
 (999999.7297500004, Arrhenius(A=9.504e+11, b=-1.012, E=6.93791e+07)),
 (9999997.297499998, Arrhenius(A=2.553e+19, b=-3.074, E=9.47048e+07))]

In [6]:
# def update_arrhenius_rate(index):
#     R = gas.reaction(index)
#     new_A = R.rate.pre_exponential_factor * 1.01
#     b = R.rate.temperature_exponent
#     E = R.rate.activation_energy
#     rate = ct.Arrhenius(new_A, b, E)
#     return rate

In [7]:
for rxn_idx in sens_indices:
    for i in rxn_idx:
        if gas.reaction(i).reaction_type == 'elementary':
            R = gas.reaction(i)
            new_A = R.rate.pre_exponential_factor * 1.01
            b = R.rate.temperature_exponent
            E = R.rate.activation_energy
            rate = ct.Arrhenius(new_A, b, E)
            R.rate = rate
            gas.modify_reaction(i, R)
        elif gas.reaction(i).reaction_type == 'falloff':
            R = gas.reaction(i)
            new_high_A = R.high_rate.pre_exponential_factor * 1.01
            high_b = R.high_rate.temperature_exponent
            high_E = R.high_rate.activation_energy
            new_high_rate = ct.Arrhenius(new_high_A, high_b, high_E)
            new_low_A = R.low_rate.pre_exponential_factor * 1.01
            low_b = R.low_rate.temperature_exponent
            low_E = R.low_rate.activation_energy
            new_low_rate = ct.Arrhenius(new_low_A, low_b, low_E)
            R.high_rate = new_high_rate
            R.low_rate = new_low_rate
            gas.modify_reaction(i, R)
        elif gas.reaction(i).reaction_type == 'pressure-dependent-Arrhenius':
            R = gas.reaction(i)
            new_rates = []
            for rate in R.rates:
                new_A = rate[1].pre_exponential_factor * 1.01
                b = rate[1].temperature_exponent
                E = rate[1].activation_energy
                new_rate = ct.Arrhenius(new_A, b, E)
                new_rates.append((rate[0], new_rate))
            R.rates = new_rates
            gas.modify_reaction(i, R)
        elif gas.reaction(i).reaction_type == 'Blowers-Masel':
            R = gas.reaction(i)
            new_A = R.rate.pre_exponential_factor * 1.01
            n = R.rate.temperature_exponent
            E0 = R.rate.intrinsic_activation_energy
            b = R.rate.bond_energy
            new_rate = ct.BlowersMasel(new_A, n, E0,b)
            R.rate = new_rate
            gas.modify_reaction(i, R)


14641.779275060566
14788.19706781117
14641.77924757891
14788.197040054698


In [4]:
def ignitionDelay(df, species):
    """
    This function computes the ignition delay from the occurence of the
    peak in species' concentration.
    """
    return df[species].idxmax()

In [6]:
# Define the reactor temperature and pressure
reactorTemperature = 1000 #Kelvin
reactorPressure = 8*101325.0 #Pascals

gas.TP = reactorTemperature, reactorPressure

# Define the fuel, oxidizer and set the stoichiometry
gas.set_equivalence_ratio(phi=1.0, fuel='C7H16(1)', oxidizer={'O2(2)':1.0, 'N2':3.76})

# Create a batch reactor object and add it to a reactor network
# In this example, the batch reactor will be the only reactor
# in the network
r = ct.IdealGasReactor(contents=gas, name='Batch Reactor')
reactorNetwork = ct.ReactorNet([r])

# Add thermo sensitivity analysis
for i in range(gas.n_reactions):
    r.add_sensitivity_reaction(i)
    
stateVariableNames = []
# now compile a list of all variables for which we will store data
stateVariableNames += [r.component_name(item) for item in range(r.n_vars)]

for rxn in gas.reactions():
    stateVariableNames.append(f"{rxn.equation} sensitivity to OH")

# use the above list to create a DataFrame
timeHistory = pd.DataFrame(columns=stateVariableNames)

In [8]:
#Tic
t0 = time.time()

# This is a starting estimate. If you do not get an ignition within this time, increase it
estimatedIgnitionDelayTime = 1
t = 0

counter = 1;
while(t < estimatedIgnitionDelayTime):
    t = reactorNetwork.step()
    if (counter%10 == 0):
        # We will save only every 10th value. Otherwise, this takes too long
        # Note that the species concentrations are mass fractions
        gas_state = reactorNetwork.get_state()
#         timeHistory.loc[t] = reactorNetwork.get_state()
        gas_sensitivities = []
        for i in range(gas.n_reactions):
            reactions_sensitivities = [reactorNetwork.sensitivity('OH(15)', i)]
            gas_sensitivities += reactions_sensitivities
        data = list(gas_state) + gas_sensitivities
#         print(data)
        timeHistory.loc[t] = data
    counter+=1

# We will use the 'oh' species to compute the ignition delay
tau = ignitionDelay(timeHistory, 'OH(15)')

#Toc
t1 = time.time()

print('Computed Ignition Delay: {:.3e} seconds. Took {:3.2f}s to compute'.format(tau, t1-t0))

# If you want to save all the data - molefractions, temperature, pressure, etc
# uncomment the next line
timeHistory.to_csv("time_history_normal_nh_kinetics_sens.csv")

KeyboardInterrupt: 

## code for script

In [ ]:
from __future__ import division
from __future__ import print_function
import yaml
import cantera as ct
import numpy as np
import pandas as pd
import time
import math

import cantera as ct
print('Runnning Cantera version: ' + ct.__version__)

# These are the sensitive reactions that we will tweak
sensitive_rxns = ["H2O2(18) (+M) <=> 2 OH(15) (+M)", "CH4(10) + O2(2) <=> CH3(19) + HO2(17)",
                 "C7H15(686) + H2O2(18) <=> C7H16(1) + HO2(17)", "C2H4(11) + OH(15) <=> C2H3(23) + H2O(8)",
                 "C2H4(11) + HO2(17) <=> C2H5(34) + O2(2)", "C7H15(683) + H2O2(18) <=> C7H16(1) + HO2(17)",
                 "2 HO2(17) <=> H2O2(18) + O2(2)", "HO2(17) + OH(15) <=> H2O(8) + O2(2)",
                 "CH3(19) + HO2(17) <=> CH3O(31) + OH(15)"
                ]
      
gas = ct.Solution("/home/xu.chao/bm_test/n_heptane/n_heptane_bm.yaml")

# extract the reaction indices
sens_indices = []
for r in sensitive_rxns:
    duplicate_rxns = []
    for i, rxn in enumerate(gas.reactions()):
        if rxn.equation == r:
            duplicate_rxns.append(i)
    sens_indices.append(duplicate_rxns)
print(sens_indices)
                
def ignitionDelay(df, species):
    """
    This function computes the ignition delay from the occurence of the
    peak in species' concentration.
    """
    return df[species].idxmax()

# tweak the kinetics parameter
for rxn_idx in sens_indices:
    for i in rxn_idx:
        if gas.reaction(i).reaction_type == 'elementary':
            R = gas.reaction(i)
            new_A = R.rate.pre_exponential_factor * 1.01
            b = R.rate.temperature_exponent
            E = R.rate.activation_energy
            rate = ct.Arrhenius(new_A, b, E)
            R.rate = rate
            gas.modify_reaction(i, R)
        elif gas.reaction(i).reaction_type == 'falloff':
            R = gas.reaction(i)
            new_high_A = R.high_rate.pre_exponential_factor * 1.01
            high_b = R.high_rate.temperature_exponent
            high_E = R.high_rate.activation_energy
            new_high_rate = ct.Arrhenius(new_high_A, high_b, high_E)
            new_low_A = R.low_rate.pre_exponential_factor * 1.01
            low_b = R.low_rate.temperature_exponent
            low_E = R.low_rate.activation_energy
            new_low_rate = ct.Arrhenius(new_low_A, low_b, low_E)
            R.high_rate = new_high_rate
            R.low_rate = new_low_rate
            gas.modify_reaction(i, R)
        elif gas.reaction(i).reaction_type == 'pressure-dependent-Arrhenius':
            R = gas.reaction(i)
            new_rates = []
            for rate in R.rates:
                new_A = rate[1].pre_exponential_factor * 1.01
                b = rate[1].temperature_exponent
                E = rate[1].activation_energy
                new_rate = ct.Arrhenius(new_A, b, E)
                new_rates.append((rate[0], new_rate))
            R.rates = new_rates
            gas.modify_reaction(i, R)